In [3]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [4]:
!pip install stable-baselines3[extra]
!pip install pyglet
#pip install tensorflow
#!pip install tensorboard #should not be necessary

In [5]:
import os
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

# 2. Load Environment

In [6]:
environment_name = "CartPole-v0"

In [7]:
env = gym.make(environment_name)

In [8]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:61.0
Episode:2 Score:35.0
Episode:3 Score:19.0
Episode:4 Score:18.0
Episode:5 Score:14.0


# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [9]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

1

In [10]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([ 2.4407070e+00, -1.0091513e+38,  3.8157588e-01,  2.7624153e+38],
      dtype=float32)

# 3. Train an RL Model

In [11]:
#make directories first
log_path = os.path.join('Training', 'Logs')
log_path = os.path.join('Training', 'Saved Models')

In [12]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
#model = PPO('MlpPolicy', env, verbose = 1)
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [13]:
model.learn(total_timesteps=20000)  #play around to higher the score - see OneNote


Logging to Training\Saved Models\PPO_5
-----------------------------
| time/              |      |
|    fps             | 311  |
|    iterations      | 1    |
|    time_elapsed    | 6    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 437         |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008770036 |
|    clip_fraction        | 0.098       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.000484   |
|    learning_rate        | 0.0003      |
|    loss                 | 6.25        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0153     |
|    value_loss           | 48.5        |
-------------------------------------

In [14]:
model

# 4. Save and Reload Model

In [15]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')

In [16]:
model.save(PPO_path)

In [17]:
#del model

In [18]:
#PPO_path

In [19]:
#model = PPO.load(PPO_path, env=env)

# 4. Evaluation

In [20]:
# from stable_baselines3.common.evaluation import evaluate_policy  #method, to evaluated, how well it performed

In [21]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

c:\Users\PeterMeyerDr\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [22]:
env.close()

# 5. Test Model

In [23]:
# my code
# 
# episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action,_ = model.predict(obs) # using the model here
        obs, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
#env.close()

Episode:1 Score:[200.]
Episode:2 Score:[200.]
Episode:3 Score:[105.]
Episode:4 Score:[116.]
Episode:5 Score:[200.]


In [24]:
# original code from Nicholas


obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

info [{'TimeLimit.truncated': True, 'terminal_observation': array([ 0.62997305,  1.1277288 , -0.15851179, -0.81423295], dtype=float32)}]


In [25]:
model.predict(obs)

(array([1], dtype=int64), None)

In [26]:
#to get the reward
action,_ = model.predict(obs)
env.step(action)

(array([[-0.01049071,  0.18271291, -0.03929755, -0.26971504]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{}])

In [27]:
env.close()

# 6. Viewing Logs in Tensorboard

In [28]:
training_log_path = os.path.join(log_path, 'PPO_3')

In [29]:
#run it outside notebook PM 290922 - something wrong with path

# !tensorboard --logdir={training_log_path}  

# 7. Adding a callback to the training Stage

In [30]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [31]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [32]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [33]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [34]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [35]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_3
-----------------------------
| time/              |      |
|    fps             | 1598 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 1095       |
|    iterations           | 2          |
|    time_elapsed         | 3          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00874511 |
|    clip_fraction        | 0.104      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.686     |
|    explained_variance   | 0.00232    |
|    learning_rate        | 0.0003     |
|    loss                 | 7.53       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0177    |
|    value_loss           | 50.7       |
----------------------------------------
---------------------

In [36]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [37]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [38]:
env.close()

# 8. Changing Policies

In [39]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [40]:
# Original by Nich

#model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

In [41]:
# PM 290922

model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = log_path, policy_kwargs={'net_arch': net_arch})

Using cpu device


In [42]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_4
-----------------------------
| time/              |      |
|    fps             | 1040 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 769         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014894287 |
|    clip_fraction        | 0.202       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.682      |
|    explained_variance   | -0.00259    |
|    learning_rate        | 0.0003      |
|    loss                 | 4.06        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0249     |
|    value_loss           | 23.5        |
-----------------------------------------
---

# 9. Using an Alternate Algorithm

In [43]:
from stable_baselines3 import DQN

In [44]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [45]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.968    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 3283     |
|    time_elapsed     | 0        |
|    total_timesteps  | 68       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.912    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 4546     |
|    time_elapsed     | 0        |
|    total_timesteps  | 185      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.873    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 5305     |
|    time_elapsed     | 0        |
|    total_timesteps  | 268      |
----------------------------------
------------------------

In [46]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [47]:
model.save(dqn_path)

In [48]:
model = DQN.load(dqn_path, env=env)

In [49]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(9.2, 0.8717797887081348)

In [ ]:
env.close()